In [285]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from itertools import islice
import time
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [286]:
def change_name_discipline(cell):
    # Функция для установки заглавной буквы для содержимого ячейки
    #очищаем от пробельных символов спереди и сзади
    value = cell.strip()
    # Делаем заглавным первый символ
    out_value = f'{value[0].upper()}{value[1:]}'

    return out_value
    

In [287]:
out_df

,Ф.И.О. преподавателя (полностью),Занимаемая в ПОО должность,Квалификационная категория,Учебная дисциплина,Курс,Группа,Теория,ЛПЗ,Учебная практика,Производственная практика,Преддипломная практика,Руководство над курсовым проектом,Консультации,"Контроль (экзамены, зачеты и т.д.)",Руководство ВКР,ИТОГО,Итого по тарификации
0,Алексеева,,,МДК 02.01. СУОТ и ПБ в горной орг,,4ОГР19б,65.0,58.0,NaN,36.0,NaN,NaN,NaN,10.0,NaN,133.0,
1,Алексеева,,,МДК 02.01. СУОТ и ПБ в горной орг,,4ОГР18а,65.0,58.0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,133.0,
2,Алексеева,,,МДК 03.03 Система упр охраной труда,,4ГСП18а,24.0,20.0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,54.0,
3,Алексеева,,,Основы проектной деятельности,,1ПСО21а,38.0,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,
4,Алексеева,,,Основы проектной деятельности,,1МД21а-2,38.0,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,
5,Алексеева,,,Основы проектной деятельности,,1ОГР21а-2,38.0,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,
6,Алексеева,,,Охрана труда,,3ХИМ19а,70.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,82.0,
7,Алексеева,,,Охрана труда,,3ТЭО19а,30.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0,
8,Алексеева,,,Охрана труда,,3ОГР19а,14.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,
9,Алексеева,,,Охрана труда,,3 ПРМ20б,40.0,31.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,79.0,


In [288]:
path_to_end_folder = 'data/'
path_data_folder = 'data/tarif_data/'


In [294]:
# Создаем общий датафрейм
col_out_df = ['Ф.И.О. преподавателя (полностью)','Занимаемая в ПОО должность','Квалификационная категория','Учебная дисциплина',
                 'Курс','Группа','Теория','ЛПЗ','Учебная практика','Производственная практика','Преддипломная практика','Руководство над курсовым проектом'
                 ,'Консультации','Контроль (экзамены, зачеты и т.д.)','Руководство ВКР','ИТОГО','Итого по тарификации']
out_df= pd.DataFrame(columns=col_out_df)


# Обрабатываем файл, пропускаем все не xlsx файлы и временные файлы
for file in os.listdir(path_data_folder):
    if not file.startswith('~') and file.endswith('.xlsx'):
        # Создаем датафрейм для соединения результата обработки таблицы и строки с суммой
        finish_df = pd.DataFrame()
        df = pd.read_excel(f'{path_data_folder}{file}',skiprows=6)
        df.columns = ['№/пп','Наименование группы','Наименование дисциплины','1 семестр кол 1','1 семестр кол 2','2 семестр кол 1','2 семестр кол 2','Всего часов',
'Теория','ЛПЗ','Учебная практика','Производственная практика','Преддипломная практика','Руководство над курсовым проектом','Консультации','Контроль (экзамены, зачеты и т.д.)',
 'Руководство ВКР','ИТОГО','Преподаватель','Промежуточные суммы']

        df = df[df['Наименование группы'].notna()]



        # Очищаем от пробелов перед и после слов
        df['Наименование группы'] =df['Наименование группы'].apply(lambda x:x.strip())
        df['Наименование дисциплины'] = df['Наименование дисциплины'].apply(lambda x:change_name_discipline(x) if type(x) == str else x)


        df = df[df['Наименование группы'] !='ознакомлен']

        # Удаляем лишний столбец
        df.drop(columns=['Промежуточные суммы'],inplace =True)
        # Создаем книгу для того чтобы отобрать все строки до внебюджета
        wb = openpyxl.Workbook()
        ws = wb.active

        for r in dataframe_to_rows(df, index=True, header=True):
            if 'внебюджет'  in r:
                break
            ws.append(r)

        # Загружаем обратно очищенный от внебюджетных дисциплин датафрейм
        data = ws.values
        cols = next(data)[1:]
        data = list(data)
        idx = [r[0] for r in data]
        data = (islice(r, 1, None) for r in data)
        clear_df = pd.DataFrame(data, index=idx, columns=cols)


        clear_df.sort_values(by='Наименование дисциплины',inplace=True)
        
        # Копируем данные  в датафрейм
        finish_df['Ф.И.О. преподавателя (полностью)'] = clear_df['Преподаватель']
        finish_df['Занимаемая в ПОО должность'] = ''
        finish_df['Квалификационная категория'] = ''
        finish_df['Учебная дисциплина'] = clear_df['Наименование дисциплины']
        finish_df['Курс'] = ''
        finish_df['Группа'] = clear_df['Наименование группы']
        finish_df['Теория'] = clear_df['Теория']
        finish_df['ЛПЗ'] = clear_df['ЛПЗ']
        finish_df['Учебная практика'] = clear_df['Учебная практика']
        finish_df['Производственная практика'] = clear_df['Производственная практика']
        finish_df['Преддипломная практика'] = clear_df['Преддипломная практика']
        finish_df['Руководство над курсовым проектом'] = clear_df['Руководство над курсовым проектом']
        finish_df['Консультации'] = clear_df['Консультации']
        finish_df['Контроль (экзамены, зачеты и т.д.)'] = clear_df['Контроль (экзамены, зачеты и т.д.)']
        finish_df['Руководство ВКР'] = clear_df['Руководство ВКР']
        finish_df['ИТОГО'] = clear_df['ИТОГО']
        finish_df['Итого по тарификации'] = ''





        finish_df.dropna(subset=['Ф.И.О. преподавателя (полностью)'],inplace=True)
        
        # Получаем сумму колонок
        sum_col = finish_df.sum(axis=0,numeric_only=True).to_frame().T
        sum_col['Ф.И.О. преподавателя (полностью)'] = 'Итого'




        finish_df=pd.concat([finish_df,sum_col],ignore_index=True)
        finish_df
        
        out_df= pd.concat([out_df,finish_df],ignore_index=True)
  


        
        


,Ф.И.О. преподавателя (полностью),Занимаемая в ПОО должность,Квалификационная категория,Учебная дисциплина,Курс,Группа,Теория,ЛПЗ,Учебная практика,Производственная практика,Преддипломная практика,Руководство над курсовым проектом,Консультации,"Контроль (экзамены, зачеты и т.д.)",Руководство ВКР,ИТОГО,Итого по тарификации
0,Алексеева,,,МДК 02.01. СУОТ и ПБ в горной орг,,4ОГР19б,65.0,58.0,NaN,36.0,NaN,NaN,NaN,10.0,NaN,133.0,
1,Алексеева,,,МДК 02.01. СУОТ и ПБ в горной орг,,4ОГР18а,65.0,58.0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,133.0,
2,Алексеева,,,МДК 03.03 Система упр охраной труда,,4ГСП18а,24.0,20.0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,54.0,
3,Алексеева,,,Основы проектной деятельности,,1ПСО21а,38.0,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,
4,Алексеева,,,Основы проектной деятельности,,1МД21а-2,38.0,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,
5,Алексеева,,,Основы проектной деятельности,,1ОГР21а-2,38.0,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,
6,Алексеева,,,Охрана труда,,3ХИМ19а,70.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,82.0,
7,Алексеева,,,Охрана труда,,3ТЭО19а,30.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0,
8,Алексеева,,,Охрана труда,,3ОГР19а,14.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,
9,Алексеева,,,Охрана труда,,3 ПРМ20б,40.0,31.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,79.0,


,Ф.И.О. преподавателя (полностью),Занимаемая в ПОО должность,Квалификационная категория,Учебная дисциплина,Курс,Группа,Теория,ЛПЗ,Учебная практика,Производственная практика,Преддипломная практика,Руководство над курсовым проектом,Консультации,"Контроль (экзамены, зачеты и т.д.)",Руководство ВКР,ИТОГО,Итого по тарификации
0,Баторова,,,Иностранный язык,,1ОПИ21а,14.0,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117.0,
1,Баторова,,,Иностранный язык,,1ОГР21а,14.0,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117.0,
2,Баторова,,,Иностранный язык,,1ХИМ21а,14.0,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117.0,
3,Баторова,,,Иностранный язык,,3 ПРМ20б,NaN,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.0,
4,Баторова,,,Иностранный язык,,3 РИПК19а,NaN,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,
5,Баторова,,,Иностранный язык,,4ОГР18а,NaN,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0,
6,Баторова,,,Иностранный язык,,4ОГР19б,NaN,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0,
7,Баторова,,,Иностранный язык,,4МД18а,NaN,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,
8,Баторова,,,Иностранный язык,,4МД19б,NaN,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,
9,Баторова,,,Иностранный язык,,4ГИГ18а,NaN,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0,


,Ф.И.О. преподавателя (полностью),Занимаемая в ПОО должность,Квалификационная категория,Учебная дисциплина,Курс,Группа,Теория,ЛПЗ,Учебная практика,Производственная практика,Преддипломная практика,Руководство над курсовым проектом,Консультации,"Контроль (экзамены, зачеты и т.д.)",Руководство ВКР,ИТОГО,Итого по тарификации
0,Башелханов,,,Административное право,,2ПСО20а,46.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,
1,Башелханов,,,ДП,,3ПСО18а,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192.0,192.0,
2,Башелханов,,,Конституционное право,,2ПСО20а,46.0,18.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,72.0,
3,Башелханов,,,МДК 01.01 право социального обеспечения,,3ПСО18а,50.0,74.0,NaN,NaN,NaN,20.0,NaN,24.0,NaN,168.0,
4,Башелханов,,,МДК 02.01 организация работы органов и учр соц...,,2ПСО20а,47.0,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,
5,Башелханов,,,МДК 02.03 правовые основы медико - социальной ...,,3ПСО18а,35.0,30.0,NaN,NaN,NaN,NaN,NaN,12.0,NaN,77.0,
6,Башелханов,,,Обществознание,,1ПСО21а,40.0,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.0,
7,Башелханов,,,Основы философии,,2ИС20а,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,
8,Башелханов,,,Основы философии,,3ХИМ19а,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,
9,Башелханов,,,Основы философии,,3 РИПК19а,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,


In [295]:
# Создаем книгу для итогового файла
out_wb = openpyxl.Workbook()
out_ws=out_wb.active


# Записываем финальный датафрейм в созданную книгу
for r in dataframe_to_rows(out_df,index=False, header=True):
    if len(r) != 1:
        out_ws.append(r)
    

t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
# Сохраняем итоговый файл
out_wb.save(f'{path_to_end_folder}Приложение №6 от {current_time}.xlsx')

In [291]:
# Обрабатываем файл, пропускаем все не xlsx файлы и временные файлы
for file in os.listdir(path_data_folder):
    if not file.startswith('~') and file.endswith('.xlsx'):
        # Создаем датафрейм для соединения результата обработки таблицы и строки с суммой
        finish_df = pd.DataFrame()
        df = pd.read_excel(f'{path_data_folder}{file}')
        df.columns = ['№/пп','Наименование группы','Наименование дисциплины','1 семестр кол 1','1 семестр кол 2','2 семестр кол 1','2 семестр кол 2','Всего часов',
'Теория','ЛПЗ','Учебная практика','Производственная практика','Преддипломная практика','Руководство над курсовым проектом','Консультации','Контроль (экзамены, зачеты и т.д.)',
 'Руководство ВКР','ИТОГО','Преподаватель','Промежуточные суммы']

        df = df[df['Наименование группы'].notna()]



        # Очищаем от пробелов перед и после слов
        df['Наименование группы'] =df['Наименование группы'].apply(lambda x:x.strip())
        df['Наименование дисциплины'] = df['Наименование дисциплины'].apply(lambda x:change_name_discipline(x) if type(x) == str else x)


        df = df[df['Наименование группы'] !='ознакомлен']

        # Удаляем лишний столбец
        df.drop(columns=['Промежуточные суммы'],inplace =True)

        wb = openpyxl.Workbook()
        ws = wb.active

        for r in dataframe_to_rows(df, index=True, header=True):
            if 'внебюджет'  in r:
                break
            ws.append(r)

        # Загружаем обратно очищенный от внебюджетных дисциплин датафрейм
        data = ws.values
        cols = next(data)[1:]
        data = list(data)
        idx = [r[0] for r in data]
        data = (islice(r, 1, None) for r in data)
        clear_df = pd.DataFrame(data, index=idx, columns=cols)


        clear_df.sort_values(by='Наименование дисциплины',inplace=True)


        finish_df['Ф.И.О. преподавателя (полностью)'] = clear_df['Преподаватель']
        finish_df['Занимаемая в ПОО должность'] = ''
        finish_df['Квалификационная категория'] = ''
        finish_df['Учебная дисциплина'] = clear_df['Наименование дисциплины']
        finish_df['Курс'] = ''
        finish_df['Группа'] = clear_df['Наименование группы']
        finish_df['Теория'] = clear_df['Теория']
        finish_df['ЛПЗ'] = clear_df['ЛПЗ']
        finish_df['Учебная практика'] = clear_df['Учебная практика']
        finish_df['Производственная практика'] = clear_df['Производственная практика']
        finish_df['Преддипломная практика'] = clear_df['Преддипломная практика']
        finish_df['Руководство над курсовым проектом'] = clear_df['Руководство над курсовым проектом']
        finish_df['Консультации'] = clear_df['Консультации']
        finish_df['Контроль (экзамены, зачеты и т.д.)'] = clear_df['Контроль (экзамены, зачеты и т.д.)']
        finish_df['Руководство ВКР'] = clear_df['Руководство ВКР']
        finish_df['ИТОГО'] = clear_df['ИТОГО']
        finish_df['Итого по тарификации'] = ''





        finish_df.dropna(subset=['Ф.И.О. преподавателя (полностью)'],inplace=True)

        # Создаем строку с суммой по колонкам
        sum_col = finish_df.sum(axis=0,numeric_only=True).to_frame().T



        finish_df=pd.concat([finish_df,sum_col])

        out_df=pd.concat([out_df,finish_df])


In [292]:
out_df.shape

(107, 17)

In [293]:
# Создаем книгу для итогового файла
out_wb = openpyxl.Workbook()
out_ws=out_wb.active


# Записываем финальный датафрейм в созданную книгу
for r in dataframe_to_rows(out_df, header=True):
    if len(r) != 1:
        out_ws.append(r)
    

t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
# Сохраняем итоговый файл
out_wb.save(f'{path_to_end_folder}Приложение №6 от {current_time}.xlsx')

df.columns = ['№/пп','Наименование группы','Наименование дисциплины','1 семестр кол 1','1 семестр кол 2','2 семестр кол 1','2 семестр кол 2','Всего часов',
'Теория','ЛПЗ','Учебная практика','Производственная практика','Преддипломная практика','Руководство над курсовым проектом','Консультации','Контроль (экзамены, зачеты и т.д.)',
 'Руководство ВКР','ИТОГО','Преподаватель','Промежуточные суммы']

df = df[df['Наименование группы'].notna()]



# Очищаем от пробелов перед и после слов
df['Наименование группы'] =df['Наименование группы'].apply(lambda x:x.strip())
df['Наименование дисциплины'] = df['Наименование дисциплины'].apply(lambda x:change_name_discipline(x) if type(x) == str else x)


df = df[df['Наименование группы'] !='ознакомлен']

# Удаляем лишний столбец
df.drop(columns=['Промежуточные суммы'],inplace =True)

wb = openpyxl.Workbook()
ws = wb.active

for r in dataframe_to_rows(df, index=True, header=True):
    if 'внебюджет'  in r:
        break

    ws.append(r)
    



# Загружаем обратно очищенный от внебюджетных дисциплин датафрейм
data = ws.values
cols = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)
clear_df = pd.DataFrame(data, index=idx, columns=cols)

clear_df.columns

clear_df.sort_values(by='Наименование дисциплины',inplace=True)


finish_df['Ф.И.О. преподавателя (полностью)'] = clear_df['Преподаватель']
finish_df['Занимаемая в ПОО должность'] = ''
finish_df['Квалификационная категория'] = ''
finish_df['Учебная дисциплина'] = clear_df['Наименование дисциплины']
finish_df['Курс'] = ''
finish_df['Группа'] = clear_df['Наименование группы']
finish_df['Теория'] = clear_df['Теория']
finish_df['ЛПЗ'] = clear_df['ЛПЗ']
finish_df['Учебная практика'] = clear_df['Учебная практика']
finish_df['Производственная практика'] = clear_df['Производственная практика']
finish_df['Преддипломная практика'] = clear_df['Преддипломная практика']
finish_df['Руководство над курсовым проектом'] = clear_df['Руководство над курсовым проектом']
finish_df['Консультации'] = clear_df['Консультации']
finish_df['Контроль (экзамены, зачеты и т.д.)'] = clear_df['Контроль (экзамены, зачеты и т.д.)']
finish_df['Руководство ВКР'] = clear_df['Руководство ВКР']
finish_df['ИТОГО'] = clear_df['ИТОГО']
finish_df['Итого по тарификации'] = ''





finish_df.dropna(subset=['Ф.И.О. преподавателя (полностью)'],inplace=True)

# Создаем строку с суммой по колонкам
sum_col = finish_df.sum(axis=0,numeric_only=True).to_frame().T

sum_col

finish_df=pd.concat([finish_df,sum_col])

out_df=pd.concat([out_df,finish_df])



# Создаем книгу для итогового файла
out_wb = openpyxl.Workbook()
out_ws=out_wb.active


# Записываем финальный датафрейм в созданную книгу
for r in dataframe_to_rows(out_df, header=True):
    if len(r) != 1:
        out_ws.append(r)
    

t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
# Сохраняем итоговый файл
out_wb.save(f'{path_to_end_folder}Приложение №6 от {current_time}.xlsx')



#df = pd.read_excel('Баторова.xlsx',skiprows=6)
df = pd.read_excel(f'{path_data_folder}Алексеева.xlsx',skiprows=6)

df.columns = ['№/пп','Наименование группы','Наименование дисциплины','1 семестр кол 1','1 семестр кол 2','2 семестр кол 1','2 семестр кол 2','Всего часов',
'Теория','ЛПЗ','Учебная практика','Производственная практика','Преддипломная практика','Руководство над курсовым проектом','Консультации','Контроль (экзамены, зачеты и т.д.)',
 'Руководство ВКР','ИТОГО','Преподаватель','Промежуточные суммы']

df = df[df['Наименование группы'].notna()]



# Очищаем от пробелов перед и после слов
df['Наименование группы'] =df['Наименование группы'].apply(lambda x:x.strip())
df['Наименование дисциплины'] = df['Наименование дисциплины'].apply(lambda x:change_name_discipline(x) if type(x) == str else x)


df = df[df['Наименование группы'] !='ознакомлен']

# Удаляем лишний столбец
df.drop(columns=['Промежуточные суммы'],inplace =True)

wb = openpyxl.Workbook()
ws = wb.active

for r in dataframe_to_rows(df, index=True, header=True):
    if 'внебюджет'  in r:
        break

    ws.append(r)
    

# Загружаем обратно очищенный от внебюджетных дисциплин датафрейм
data = ws.values
cols = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)
clear_df = pd.DataFrame(data, index=idx, columns=cols)

clear_df.columns

clear_df.sort_values(by='Наименование дисциплины',inplace=True)


finish_df['Ф.И.О. преподавателя (полностью)'] = clear_df['Преподаватель']
finish_df['Занимаемая в ПОО должность'] = ''
finish_df['Квалификационная категория'] = ''
finish_df['Учебная дисциплина'] = clear_df['Наименование дисциплины']
finish_df['Курс'] = ''
finish_df['Группа'] = clear_df['Наименование группы']
finish_df['Теория'] = clear_df['Теория']
finish_df['ЛПЗ'] = clear_df['ЛПЗ']
finish_df['Учебная практика'] = clear_df['Учебная практика']
finish_df['Производственная практика'] = clear_df['Производственная практика']
finish_df['Преддипломная практика'] = clear_df['Преддипломная практика']
finish_df['Руководство над курсовым проектом'] = clear_df['Руководство над курсовым проектом']
finish_df['Консультации'] = clear_df['Консультации']
finish_df['Контроль (экзамены, зачеты и т.д.)'] = clear_df['Контроль (экзамены, зачеты и т.д.)']
finish_df['Руководство ВКР'] = clear_df['Руководство ВКР']
finish_df['ИТОГО'] = clear_df['ИТОГО']
finish_df['Итого по тарификации'] = ''





finish_df.dropna(subset=['Ф.И.О. преподавателя (полностью)'],inplace=True)

# Создаем строку с суммой по колонкам
sum_col = finish_df.sum(axis=0,numeric_only=True).to_frame().T

sum_col

finish_df=pd.concat([finish_df,sum_col])

out_df=pd.concat([out_df,finish_df])

# Создаем книгу для итогового файла
out_wb = openpyxl.Workbook()
out_ws=out_wb.active


# Записываем финальный датафрейм в созданную книгу
for r in dataframe_to_rows(out_df, header=True):
    if len(r) != 1:
        out_ws.append(r)
    

t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
# Сохраняем итоговый файл
out_wb.save(f'{path_to_end_folder}Приложение №6 от {current_time}.xlsx')